In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3

In [2]:
db_path = 'C:/Users/rashi/ESM_databases/temoa/data_files/'
db_name = 'canoe_on_12d_lifetimes' + '.sqlite'

In [3]:
db_file = db_path + db_name

conn = sqlite3.connect(db_file)
query_net_capacity = "SELECT * FROM OutputNetCapacity WHERE sector = 'Transport'"
query_built_capacity = "SELECT * FROM OutputBuiltCapacity WHERE sector = 'Transport'"
query_existing_capacity = "SELECT * FROM ExistingCapacity WHERE tech like 'T_%'"

net_cap = pd.read_sql_query(query_net_capacity, conn).drop(columns=['scenario', 'region', 'sector'])
new_cap = pd.read_sql_query(query_built_capacity, conn).drop(columns=['scenario', 'region', 'sector'])
ex_cap =  pd.read_sql_query(query_existing_capacity, conn)[['tech', 'vintage', 'capacity', 'units']]

conn.close()

In [4]:
class_mapping = {
    'T_HDV_AJ': 'Air jet',
    'T_HDV_B': 'Bus',
    'T_HDV_R': 'Rail',
    'T_HDV_T': 'HD Truck',
    'T_HDV_W': 'Marine vessel',
    'T_MDV_T': 'MD Truck',
    'T_LDV_C_': 'LD Car',
    'T_LDV_LT': 'LD Truck',
    'T_LDV_M': 'Motorcycle',
    'T_IMP_': 'Fuel use',
    'H2_distribution': 'Fuel use'
}

def map_tech_to_mode(tech):
    for prefix, class_name in class_mapping.items():
        if tech.startswith(prefix):
            return class_name
    return 'Other'

carrier_mapping = {
        'BEV': 'BEV',
        'GSL': 'Gasoline',
        'DSL': 'Diesel',
        'CNG': 'CNG',
        'LNG': 'LNG',
        'JTF': 'Jet Fuel',
        'SPK': 'Synthetic Jet Fuel', #      Might change for SAF
        'HFO': 'Heavy Fuel Oil',
        'MDO': 'Marine Diesel Oil',
        'H2': 'H2',
        'ELC': 'Electricity'
    }

def map_tech_to_fuel(tech):
    # Check hybrids first
    if 'PHEV35' in tech:
        return 'PHEV35'
    if 'PHEV50' in tech:
        return 'PHEV50'
    if 'PHEV' in tech:
        return 'PHEV'
    if 'BEV150' in tech:
        return 'BEV150'
    if 'BEV200' in tech:
        return 'BEV200'
    if 'BEV300' in tech:
        return 'BEV300'
    if 'BEV400' in tech:
        return 'BEV400'
    if 'FC' in tech:
        return 'FCEV'
    if 'HEV' in tech:
        return 'HEV'
    if 'RDSL' in tech:
            return 'Biodiesel' #            Might differentiate
    if 'ETH' in tech:
        return 'Ethanol' #                  Might differentiate
    for prefix, carrier in carrier_mapping.items():
        if prefix in tech:
            return carrier 
    return 'Other'

In [5]:
net_cap['mode'] = net_cap['tech'].apply(map_tech_to_mode)
new_cap['mode'] = new_cap['tech'].apply(map_tech_to_mode)
ex_cap['mode'] = ex_cap['tech'].apply(map_tech_to_mode)

net_cap['fuel'] = net_cap['tech'].apply(map_tech_to_fuel)
new_cap['fuel'] = new_cap['tech'].apply(map_tech_to_fuel)
ex_cap['fuel'] = ex_cap['tech'].apply(map_tech_to_fuel)

In [6]:
net_cap_group = net_cap.groupby(['mode', 'fuel', 'period'], as_index=False).sum('capacity').drop(columns='vintage').rename(columns={'period': 'vintage'})
net_cap_group[(net_cap_group['mode'] == 'LD Car') & (net_cap_group['fuel'] == 'Gasoline')].sort_values(by=['fuel', 'vintage'])

,mode,fuel,vintage,capacity
105,LD Car,Gasoline,2021,3601.866902
106,LD Car,Gasoline,2025,2822.689199
107,LD Car,Gasoline,2030,1729.573568
108,LD Car,Gasoline,2035,753.839412
109,LD Car,Gasoline,2040,414.556514
110,LD Car,Gasoline,2045,0.006146
111,LD Car,Gasoline,2050,0.005629


In [7]:
ex_cap_group = ex_cap.copy()
ex_cap_group['vintage'] = 2021
ex_cap_group = ex_cap_group.groupby(['mode', 'fuel', 'vintage', 'units'], as_index=False).sum('capacity')

In [8]:
new_cap_group = new_cap.groupby(['mode', 'fuel', 'vintage'], as_index=False).sum('capacity')
new_cap_group[(new_cap_group['mode'] == 'LD Car') & (new_cap_group['fuel'] == 'Gasoline')].sort_values(by=['vintage'])

,mode,fuel,vintage,capacity
90,LD Car,Gasoline,2021,0.002812
91,LD Car,Gasoline,2025,0.001944
92,LD Car,Gasoline,2030,0.001892
93,LD Car,Gasoline,2035,0.001549
94,LD Car,Gasoline,2040,0.001557
95,LD Car,Gasoline,2045,0.001765
96,LD Car,Gasoline,2050,0.001836


In [9]:
net_cap_group = net_cap_group[['mode', 'fuel', 'vintage', 'capacity']].rename(columns={'capacity': 'net capacity'})
new_cap_group = new_cap_group[['mode', 'fuel', 'vintage', 'capacity']].rename(columns={'capacity': 'new capacity'})
ex_cap_group = ex_cap_group[['mode', 'fuel', 'vintage', 'capacity']].rename(columns={'capacity': 'ex capacity'})

merged_cap = net_cap_group.merge(new_cap_group, on=['mode', 'fuel', 'vintage'], how='left').merge(ex_cap_group, on=['mode', 'fuel', 'vintage'], how='left').fillna(0.)

In [10]:
merged_cap[(merged_cap['mode'] == 'LD Car') & (merged_cap['fuel'] == 'Gasoline')].sort_values(by=['fuel', 'vintage'])

,mode,fuel,vintage,net capacity,new capacity,ex capacity
106,LD Car,Gasoline,2021,3601.866902,0.002812,3601.86409
107,LD Car,Gasoline,2025,2822.689199,0.001944,0.00000
108,LD Car,Gasoline,2030,1729.573568,0.001892,0.00000
109,LD Car,Gasoline,2035,753.839412,0.001549,0.00000
110,LD Car,Gasoline,2040,414.556514,0.001557,0.00000
111,LD Car,Gasoline,2045,0.006146,0.001765,0.00000
112,LD Car,Gasoline,2050,0.005629,0.001836,0.00000


In [11]:
merged_cap = merged_cap.sort_values(by=['mode', 'fuel', 'vintage'])
merged_cap['retired cap'] = 0.

for _, group in merged_cap.groupby(['mode', 'fuel']):
    for i in range(1, len(group)):
        current_idx = group.index[i]
        previous_idx = group.index[i-1]

        # Calculate retired capacity as existing capacity + new capacity - net capacity of previous vintage
        merged_cap.loc[current_idx, 'retired cap'] = (
            merged_cap.loc[previous_idx, 'net capacity'] + 
            merged_cap.loc[current_idx, 'new capacity'] - 
            merged_cap.loc[current_idx, 'net capacity']
        )

for _, group in merged_cap.groupby(['mode', 'fuel']):
    for i in range(1, len(group)):
        current_idx = group.index[i]
        previous_idx = group.index[i-1]

        # Fill the "ex capacity" for years after 2021 using the "net capacity" from the previous year
        merged_cap.loc[current_idx, 'ex capacity'] = merged_cap.loc[previous_idx, 'net capacity']

merged_cap['retired cap'] = -merged_cap['retired cap']

In [12]:
merged_cap[(merged_cap['mode'] == 'LD Car') & (merged_cap['fuel'] == 'BEV150')]

,mode,fuel,vintage,net capacity,new capacity,ex capacity,retired cap
57,LD Car,BEV150,2021,727.411258,686.594348,40.816910,-0.000000
58,LD Car,BEV150,2025,2876.439506,2149.937278,727.411258,-0.909030
59,LD Car,BEV150,2030,4037.376101,1343.698439,2876.439506,-182.761845
60,LD Car,BEV150,2035,6140.081511,3002.077298,4037.376101,-899.371889
61,LD Car,BEV150,2040,6754.421764,2026.142532,6140.081511,-1411.802279
62,LD Car,BEV150,2045,7325.776047,2003.018025,6754.421764,-1431.663742
63,LD Car,BEV150,2050,7553.536834,2944.467976,7325.776047,-2716.707189


In [13]:
merged_cap[(merged_cap['mode'] == 'LD Car') & (merged_cap['fuel'] == 'Gasoline')]

,mode,fuel,vintage,net capacity,new capacity,ex capacity,retired cap
106,LD Car,Gasoline,2021,3601.866902,0.002812,3601.864090,-0.000000
107,LD Car,Gasoline,2025,2822.689199,0.001944,3601.866902,-779.179647
108,LD Car,Gasoline,2030,1729.573568,0.001892,2822.689199,-1093.117523
109,LD Car,Gasoline,2035,753.839412,0.001549,1729.573568,-975.735705
110,LD Car,Gasoline,2040,414.556514,0.001557,753.839412,-339.284455
111,LD Car,Gasoline,2045,0.006146,0.001765,414.556514,-414.552133
112,LD Car,Gasoline,2050,0.005629,0.001836,0.006146,-0.002353


In [14]:
merged_cap[merged_cap['mode'] == 'LD Car'].groupby(['mode', 'vintage']).sum(['net capacity', 'new capacity', 'ex capacity', 'retired cap'])

net capacity  new capacity  ex capacity  retired cap
mode   vintage                                                      
LD Car 2021      4502.158719    686.633039  3815.525680     0.000000
       2025      5834.741455   2149.963468  4502.158719  -817.380732
       2030      5847.325209   1343.732246  5834.741455 -1331.148492
       2035      6930.909535   3002.107988  5847.325209 -1918.523662
       2040      7187.619789   2026.176842  6930.909535 -1769.466588
       2045      7325.894291   2003.054270  7187.619789 -1864.779767
       2050      7553.656396   2944.509436  7325.894291 -2716.747331

In [15]:
cap = merged_cap.melt(id_vars=['mode', 'fuel', 'vintage'],
                       value_vars=['net capacity', 'ex capacity', 'new capacity', 'retired cap'],
                       var_name='cap type', value_name='capacity')

cap['capacity'] = cap['capacity'] / 1E3
cap = cap[(abs(cap['capacity']) > 1e-3) & (
        (cap['mode'] == 'LD Car') | 
        (cap['mode'] == 'LD Truck') 
        # (cap['mode'] == 'MD Truck') |
        # (cap['mode'] == 'HD Truck')
        ) & (cap['cap type'] != 'ex capacity')].reset_index(drop=True)
cap['vintage'] = cap['vintage'].astype('str')

# cap.to_csv('car_cap.csv', index=False)
# cap.to_clipboard()

In [16]:
fig = px.bar(cap, x='cap type', y='capacity', color='fuel', 
            pattern_shape='cap type', pattern_shape_sequence=["", ".", "/"],
            facet_col='vintage', facet_col_spacing=2E-2, 
            facet_row='mode', facet_row_spacing=1E-2,
            category_orders={"cap type": ["net capacity", "new capacity", "retired cap"],
                             "fuel": ['Gasoline', 'Diesel', 'CNG', 'BEV150', 'BEV200', 'BEV300', 'BEV400', 'HEV', 'PHEV35', 'PHEV50', 'FCEV']},
            # labels={'capacity': 'Fleet capacity (M units)', 'fuel': 'Fuel type'},
            template='plotly_white', orientation='v', color_discrete_sequence=px.colors.qualitative.Bold,
            text_auto='.2f',
            width=1200, height=800
            )

dummy_traces = [
    dict(
        name='',  # Empty legend
        x=[None], 
        y=[None], 
        marker=dict(color='rgba(0,0,0,0)'),
        showlegend=True,
        legendgroup='Blank',
        legendgrouptitle=None
    ),
    # dict(
    #     name=' ',  # Another empty legend
    #     x=[None], 
    #     y=[None], 
    #     marker=dict(color='rgba(0,0,0,0)'),
    #     showlegend=True,
    #     legendgroup='Blank',
    #     legendgrouptitle=None   
    # ),
    dict(
        name='Net capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape=""
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=dict(text="Capacity type"),
    ),
    dict(
        name='New capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape="."
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=None
    ),
    dict(
        name='Retired capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape="/"
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=None
    )
]

for trace in dummy_traces:
    fig.add_bar(
        x=trace['x'], 
        y=trace['y'], 
        name=trace['name'], 
        marker=trace['marker'], 
        showlegend=trace['showlegend'],
        legendgroup=trace['legendgroup'],
        legendgrouptitle=trace['legendgrouptitle']
    )

fig.update_layout(
    margin=dict(
        t=10, b=10
    ),
    title=dict(
        text='Transport LDV fleet stock and flow in ON by vehicle class and drivetrain',
        x=0.5, 
        y=0.98,
        xanchor='center',
        yanchor='top'
    ),
    yaxis_title_standoff=0,
    legend_title_text='Fuel type',
    bargap=0.1,
    legend=dict(
        orientation='v',
        # entrywidth=50,
        yanchor='top',
        y=0.8,  
        xanchor='center',
        x=1.15),
    font=dict(
        size=15)
    )

fig.for_each_trace(lambda trace: trace.update(textfont=dict(size=11)))

for axis in fig.layout:
    if axis.startswith('xaxis'):
        fig.layout[axis].title.text = ''
        fig.layout[axis].showticklabels = False
    if axis.startswith('yaxis'):
        fig.layout[axis].title.text = ''
        # fig.layout[axis].dtick = 2

for annotation in fig.layout.annotations:
    if 'mode' in annotation.text:
        annotation.text = 'Class: ' + annotation.text.split('=')[1]
        annotation.font.size = 16
        annotation.x = 1.01
        annotation.xanchor = 'center'
    else:
        annotation.text = annotation.text.split('=')[1]
        annotation.y = 0.005
        annotation.yanchor = 'top' 

shown_legends = set()
for trace in fig.data:
    trace.name = trace.name.split(",")[0]
    if trace.name not in shown_legends:
        trace.showlegend = True
        shown_legends.add(trace.name)
    else:
        trace.showlegend = False

fig.add_annotation(
    text="Fleet capacity (M units)",
    x=-0.07,
    y=0.5,
    xref="paper",
    yref="paper",
    showarrow=False,
    textangle=-90,
    font=dict(size=17))

fig.show()

# fig.write_image("vanilla2_fleet_flow.svg", scale=1, engine='kaleido')

In [22]:
def map_tech_survival(tech):
    if '_S25' in tech:
        return 'Survival Q1'
    if '_S75' in tech:
        return 'Survival Q4'
    else: return 'Survival Q2-3'

if 'lifetime' in db_name:
    net_cap['life'] = net_cap['tech'].apply(map_tech_survival)
    new_cap['life'] = new_cap['tech'].apply(map_tech_survival)
    ex_cap['life'] = ex_cap['tech'].apply(map_tech_survival)
    net_cap_group = net_cap.groupby(['mode', 'fuel', 'life', 'period'], as_index=False).sum('capacity').drop(columns='vintage').rename(columns={'period': 'vintage'})
    ex_cap_group = ex_cap.copy()
    ex_cap_group['vintage'] = 2021
    ex_cap_group = ex_cap_group.groupby(['mode', 'fuel', 'life', 'vintage', 'units'], as_index=False).sum('capacity')
    new_cap_group = new_cap.groupby(['mode', 'fuel', 'life', 'vintage'], as_index=False).sum('capacity')
    net_cap_group = net_cap_group[['mode', 'fuel', 'life', 'vintage', 'capacity']].rename(columns={'capacity': 'net capacity'})
    new_cap_group = new_cap_group[['mode', 'fuel', 'life', 'vintage', 'capacity']].rename(columns={'capacity': 'new capacity'})
    ex_cap_group = ex_cap_group[['mode', 'fuel', 'life', 'vintage', 'capacity']].rename(columns={'capacity': 'ex capacity'})

    merged_cap = net_cap_group.merge(new_cap_group, on=['mode', 'fuel', 'life', 'vintage'], how='left').merge(ex_cap_group, on=['mode', 'fuel', 'life', 'vintage'], how='left').fillna(0.)
    merged_cap = merged_cap.sort_values(by=['mode', 'fuel', 'life', 'vintage'])
    merged_cap['retired cap'] = 0.

    for _, group in merged_cap.groupby(['mode', 'fuel', 'life']):
        for i in range(1, len(group)):
            current_idx = group.index[i]
            previous_idx = group.index[i-1]

            # Calculate retired capacity as existing capacity + new capacity - net capacity of previous vintage
            merged_cap.loc[current_idx, 'retired cap'] = (
                merged_cap.loc[previous_idx, 'net capacity'] + 
                merged_cap.loc[current_idx, 'new capacity'] - 
                merged_cap.loc[current_idx, 'net capacity']
            )

    for _, group in merged_cap.groupby(['mode', 'fuel', 'life']):
        for i in range(1, len(group)):
            current_idx = group.index[i]
            previous_idx = group.index[i-1]

            # Fill the "ex capacity" for years after 2021 using the "net capacity" from the previous year
            merged_cap.loc[current_idx, 'ex capacity'] = merged_cap.loc[previous_idx, 'net capacity']

    merged_cap['retired cap'] = -merged_cap['retired cap']

    cap = merged_cap.melt(id_vars=['mode', 'fuel', 'life', 'vintage'],
                       value_vars=['net capacity', 'ex capacity', 'new capacity', 'retired cap'],
                       var_name='cap type', value_name='capacity')

    cap['capacity'] = cap['capacity'] / 1E3
    cap = cap[(abs(cap['capacity']) > 1e-3) & (
                                                (cap['mode'] == 'LD Car') |
                                                (cap['mode'] == 'LD Truck')
                                                ) & (cap['cap type'] != 'ex capacity')].reset_index(drop=True)
    cap['vintage'] = cap['vintage'].astype('str')

cap = cap.groupby(['mode', 'life', 'vintage', 'cap type'], as_index=False).sum('capacity')
fig = px.bar(cap[cap['vintage'] != "2021"], x='cap type', y='capacity', color='life', 
            pattern_shape='cap type', pattern_shape_sequence=["", ".", "/"],
            facet_col='vintage', facet_col_spacing=2E-2, 
            facet_row='mode', facet_row_spacing=1E-2,
            category_orders={"cap type": ["net capacity", "new capacity", "retired cap"],
                             "fuel": ['Gasoline', 'Diesel', 'CNG', 'BEV200', 'BEV300', 'BEV400', 'BEV500', 'HEV', 'PHEV20', 'PHEV50', 'FCEV'],
                             "life": ['Survival Q1', 'Survival Q2-3', 'Survival Q4']},
            # labels={'capacity': 'Fleet capacity (M units)', 'fuel': 'Fuel type'},
            template='plotly_white', orientation='v', color_discrete_sequence=px.colors.qualitative.Bold,
            text_auto='.2f',
            width=1200, height=800
            )

dummy_traces = [
    dict(
        name='',  # Empty legend
        x=[None], 
        y=[None], 
        marker=dict(color='rgba(0,0,0,0)'),
        showlegend=True,
        legendgroup='Blank',
        legendgrouptitle=None
    ),
    # dict(
    #     name=' ',  # Another empty legend
    #     x=[None], 
    #     y=[None], 
    #     marker=dict(color='rgba(0,0,0,0)'),
    #     showlegend=True,
    #     legendgroup='Blank',
    #     legendgrouptitle=None   
    # ),
    dict(
        name='Net capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape=""
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=dict(text="Capacity type"),
    ),
    dict(
        name='New capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape="."
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=None
    ),
    dict(
        name='Retired capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape="/"
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=None
    )
]

for trace in dummy_traces:
    fig.add_bar(
        x=trace['x'], 
        y=trace['y'], 
        name=trace['name'], 
        marker=trace['marker'], 
        showlegend=trace['showlegend'],
        legendgroup=trace['legendgroup'],
        legendgrouptitle=trace['legendgrouptitle']
    )

fig.update_layout(
    margin=dict(
        t=10, b=10
    ),
    title=dict(
        text='Transport LDV fleet stock and flow in ON by vehicle class and lifetime',
        x=0.5, 
        y=0.97,
        xanchor='center',
        yanchor='top'
    ),
    yaxis_title_standoff=0,
    legend_title_text='Survival rate class',
    bargap=0.1,
    legend=dict(
        orientation='v',
        # entrywidth=50,
        yanchor='top',
        y=0.8,  
        xanchor='center',
        x=1.15),
    font=dict(
        size=15)
    )

fig.for_each_trace(lambda trace: trace.update(textfont=dict(size=11)))

# fig.update_xaxes(tickvals=["net capacity", "new capacity", "retired cap"],
#                  ticktext=["Net", "New", "Retired"], 
#                  tickangle=45
#                  )

for axis in fig.layout:
    if axis.startswith('xaxis'):
        fig.layout[axis].title.text = ''
        fig.layout[axis].showticklabels = False
    if axis.startswith('yaxis'):
        fig.layout[axis].title.text = ''
        fig.layout[axis].dtick = 2

for annotation in fig.layout.annotations:
    if 'mode' in annotation.text:
        annotation.text = 'Class: ' + annotation.text.split('=')[1]
        annotation.font.size = 16
        annotation.x = 1.01
        annotation.xanchor = 'center'
    else:
        annotation.text = annotation.text.split('=')[1]
        annotation.y = 0.005
        annotation.yanchor = 'top' 

shown_legends = set()
for trace in fig.data:
    trace.name = trace.name.split(",")[0]
    if trace.name not in shown_legends:
        trace.showlegend = True
        shown_legends.add(trace.name)
    else:
        trace.showlegend = False

fig.add_annotation(
    text="Fleet capacity (M units)",
    x=-0.07,
    y=0.5,
    xref="paper",
    yref="paper",
    showarrow=False,
    textangle=-90,
    font=dict(size=17))

fig.show()

fig.write_image("lifetimes_fleet_flow.svg", scale=1, engine='kaleido')